In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, recall_score
from sklearn.decomposition import PCA
from scipy.stats import ttest_ind
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV



# Ignorar warnings de convergência
import warnings

warnings.filterwarnings("ignore")




In [2]:
df = pd.read_csv("dados\dados_tratados_thiago.csv")

In [3]:
df.columns.tolist()

['idade_pessoa',
 'renda_pessoa',
 'tempo_emprego_pessoa',
 'valor_emprestimo',
 'taxa_juros_emprestimo',
 'percentual_emprestimo_renda',
 'inadimplente_arquivo_pessoa',
 'tempo_historico_credito_pessoa',
 'status_emprestimo',
 'alto_risco_comprometimento_renda',
 'risco_taxa_juros',
 'alto_risco_nota_emprestimo',
 'posse_imovel_pessoa_hipoteca',
 'posse_imovel_pessoa_proprio',
 'finalidade_emprestimo_educacao',
 'finalidade_emprestimo_empreendimento',
 'finalidade_emprestimo_pessoal',
 'finalidade_emprestimo_saude',
 'nota_emprestimo_B',
 'nota_emprestimo_C',
 'nota_emprestimo_D',
 'nota_emprestimo_E',
 'nota_emprestimo_F']

In [4]:
def clf_metrics_com_return(modelo, X, y_true, label, plot_conf_mat=True, print_cr=True):
    
    if print_cr:
        print(f"\nMétricas de avaliação de {label}:\n")
    
    y_pred = modelo.predict(X)

    if plot_conf_mat:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))

        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax[0]) 
        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, normalize="all", ax=ax[1])
        plt.show()

    if print_cr:
        print(classification_report(y_true, y_pred))
    
    return classification_report(y_true, y_pred, output_dict=True)

In [5]:
# class OutlierDetector(BaseEstimator, TransformerMixin):
#     def __init__(self, col_categoria, col_valor, faixa_de_tolerancia=1.5):
#         self.col_categoria = col_categoria
#         self.col_valor = col_valor
#         self.faixa_de_tolerancia = faixa_de_tolerancia
#         self.limites_outliers = {}

#     def fit(self, X, y=None):
#         # Calcula os limites de outliers (IQR) para cada categoria do propósito
#         for categoria in X[self.col_categoria].unique():
#             subset = X[X[self.col_categoria] == categoria][self.col_valor]
#             Q1 = subset.quantile(0.25)
#             Q3 = subset.quantile(0.75)
#             IQR = Q3 - Q1
#             limite_inferior = Q1 - self.faixa_de_tolerancia * IQR
#             limite_superior = Q3 + self.faixa_de_tolerancia * IQR
#             self.limites_outliers[categoria] = (limite_inferior, limite_superior)
#         return self

#     def transform(self, X, y=None):
#         # Cria uma coluna para marcar os outliers como 1 ou 0
#         X = X.copy()
#         X['outlier_valor_proposito'] = X.apply(
#             lambda row: 1 if (row[self.col_valor] < self.limites_outliers[row[self.col_categoria]][0] or
#                               row[self.col_valor] > self.limites_outliers[row[self.col_categoria]][1])
#             else 0, axis=1
#         )
#         return X

In [6]:
colunas_categoricas = df.drop(columns=['status_emprestimo']).select_dtypes(include=['category', 'object']).columns.tolist()
colunas_numericas = df.drop(columns=['status_emprestimo']).select_dtypes(include=['number']).columns.tolist()
# Tirei negativado pq ele também é binário (0, 1)


# Configurando o OneHotEncoder no ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop=None, sparse_output=False, handle_unknown='ignore'), colunas_categoricas),
        ('num', StandardScaler(), colunas_numericas)
    ],
    remainder='passthrough'  # Mantém as outras colunas sem mudanças
)

In [7]:
X = df.drop(columns=['status_emprestimo'], axis=1)
y = df['status_emprestimo']

X_teste, X_treino, y_teste, y_treino = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y, shuffle=True)

In [8]:
scale_pos_weight = len(df[df['status_emprestimo'] == 0]) / len(df[df['status_emprestimo'] == 1])
scale_pos_weight

6.022757216433106

In [9]:
# # Vou fazer alguns testes aqui. Tanto implementar balanceamento dentro dos hiperparâmetros dos estimadores, como SMOTE



# # Regressão logística
# # pipe_logit_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
# #                         ('preprocessor', preprocessor),
# #                        ("logit", LogisticRegression(random_state=42))])

# pipe_logit_balanceado = Pipeline([('preprocessor', preprocessor),
#                        ("logit", LogisticRegression(random_state=42, class_weight='balanced', max_iter=500, n_jobs=-1))])

# pipe_logit_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
#                        ("logit", LogisticRegression(random_state=42, max_iter=500, n_jobs=-1))])


# # Random Forest
# # pipe_rf_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
# #                         ('preprocessor', preprocessor),
# #                     ("rf", RandomForestClassifier(random_state=42))])

# pipe_rf_balanceado = Pipeline([('preprocessor', preprocessor),
#                     ("rf", RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1))])

# pipe_rf_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
#                     ("rf", RandomForestClassifier(random_state=42, n_jobs=-1))])

# # Decision Tree
# # pipe_dt_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
# #                         ('preprocessor', preprocessor),
# #                     ("dt", DecisionTreeClassifier(random_state=42))])

# pipe_dt_balanceado = Pipeline([('preprocessor', preprocessor),
#                     ("dt", DecisionTreeClassifier(random_state=42, class_weight='balanced'))])

# pipe_dt_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
#                     ("dt", DecisionTreeClassifier(random_state=42))])



# # SVM

# # pipe_svm_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
# #                          ('preprocessor', preprocessor),
# #                      ("svm", SVC(random_state=42))])

# pipe_svm_balanceado = Pipeline([('preprocessor', preprocessor),
#                      ("svm", SVC(random_state=42, class_weight='balanced'))])

# pipe_svm_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
#                      ("svm", SVC(random_state=42))])


# # Catboost 

# # pipe_catboost_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
# #                               ('preprocessor', preprocessor),
# #                               ("catboost", CatBoostClassifier(random_state=42, verbose=0))])

# pipe_catboost_balanceado = Pipeline([('preprocessor', preprocessor),
#                                      ("catboost", CatBoostClassifier(random_state=42, auto_class_weights='Balanced', verbose=0))])

# # CatBoost com SMOTE
# pipe_catboost_smote = Pipeline([#("smote", SMOTE(random_state=42, n_jobs=-1)),
#                                    ("catboost", CatBoostClassifier(random_state=42, verbose=0))])


# # XGBoost


# pipe_xbgoost = Pipeline([('preprocessor', preprocessor),
#                          ("xgboost", XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight))])

# # XGBoost com SMOTE

# pipe_xbgoost_smote = Pipeline([("xgboost", XGBClassifier(random_state=42))])


# # LGBM 

# pipe_lgbm = Pipeline([('preprocessor', preprocessor),
#                       ("lgbm", LGBMClassifier(random_state=42, is_unbalance=True))])


# # LGBM com Smote

# pipe_lgbm_smote = Pipeline([("lgbm", LGBMClassifier(random_state=42))])


# # KNN 

# pipe_knn = Pipeline([('preprocessor', preprocessor),
#                      ("knn", KNeighborsClassifier())])


# # KNN com Smote

# pipe_knn_smote = Pipeline([("knn", KNeighborsClassifier())])

# # ================================================


# dict_pipes = {"logit balanceado": pipe_logit_balanceado,
#               "random forest balanceado": pipe_rf_balanceado,
#               "decision tree balanceado": pipe_dt_balanceado,
#               "svm balanceado": pipe_svm_balanceado,
#               "catboost balancado": pipe_catboost_balanceado,
#               "XGBoost balanceado": pipe_xbgoost,
#               "LGBM balanceado": pipe_lgbm,
#               "KNN balanceado": pipe_knn_smote}

# dict_pipes_smote = {"logit smote": pipe_logit_smote,
#                     "random forest smote": pipe_rf_smote,
#                     "decision tree smote": pipe_dt_smote,
#                     "svm smote": pipe_svm_smote,
#                     "catboost smote": pipe_catboost_smote,
#                     "XGBoost smote": pipe_xbgoost_smote,
#                     "LGBM smote": pipe_lgbm_smote,
#                     "KNN Smote": pipe_knn_smote}

In [10]:
# # Mesma coisa usando CV

# # Configuração do StratifiedKFold
# kf = StratifiedKFold(n_splits=5)

# # Inicialização do dicionário de resultados
# resultado_experimentos = {"estimador": [], "recall_treino": [], "recall_teste": []}

# # Loop pelos pipelines
# for nome_modelo, pipeline in dict_pipes.items():
#     recall_treino_lista = []
#     recall_teste_lista = []
    
#     # Realiza a validação cruzada estratificada
#     for indice_treino, indice_valida in kf.split(X_treino, y_treino):
#         X_treino_split, X_valida_split = X_treino.iloc[indice_treino], X_treino.iloc[indice_valida]
#         y_treino_split, y_valida_split = y_treino.iloc[indice_treino], y_treino.iloc[indice_valida]
        
#         # Treina o modelo
#         pipeline.fit(X_treino_split, y_treino_split)
        
#         # Avalia o modelo no treino
#         y_pred_treino = pipeline.predict(X_treino_split)
#         recall_treino = recall_score(y_treino_split, y_pred_treino, average='binary')
#         recall_treino_lista.append(recall_treino)
        
#         # Avalia o modelo no teste (validação)
#         y_pred_valida = pipeline.predict(X_valida_split)
#         recall_teste = recall_score(y_valida_split, y_pred_valida, average='binary')
#         recall_teste_lista.append(recall_teste)
    
#     # Calcula a média dos recalls
#     recall_treino_medio = np.mean(recall_treino_lista)
#     recall_teste_medio = np.mean(recall_teste_lista)
    
#     # Armazena os resultados
#     resultado_experimentos["estimador"].append(nome_modelo)
#     resultado_experimentos["recall_treino"].append(recall_treino_medio)
#     resultado_experimentos["recall_teste"].append(recall_teste_medio)
    
#     print(f'Treinamento do modelo {nome_modelo} finalizado')

# # Resultado final em DataFrame
# df_resultados = pd.DataFrame(resultado_experimentos)

# # Em casos de underfit, calcula a diferença entre o recall de treino e teste
# df_resultados["gap"] = (df_resultados["recall_treino"] - df_resultados["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# # Ordena os resultados pelo recall no teste
# df_resultados = df_resultados.sort_values("recall_teste", ascending=False)

# df_resultados

In [11]:
# transformar idade em uma coluna faixa etária e dividir em bins
# Deixar para transformar no one hot enconder só depois pq enquanto ainda for faixa vai ser utilizado
# para pegar outlier de renda ou valor total do empréstimo

In [12]:
# # Mesma coisa usando CV e SMOTE

# # Configuração do StratifiedKFold
# kf = StratifiedKFold(n_splits=5)

# # Inicialização do dicionário de resultados
# resultado_experimentos = {"estimador": [], "recall_treino": [], "recall_teste": []}

# smote = SMOTE(random_state=42, n_jobs=-1)

# X = df.drop(columns=['status_emprestimo'], axis=1)
# y = df['status_emprestimo']

# X_teste, X_treino, y_teste, y_treino = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y, shuffle=True)


# # Loop pelos pipelines
# for nome_modelo, pipeline in dict_pipes_smote.items():
#     recall_treino_lista = []
#     recall_teste_lista = []
    
#     # Realiza a validação cruzada estratificada
#     for indice_treino, indice_valida in kf.split(X_treino, y_treino):
#         X_treino_split, X_valida_split = X_treino.iloc[indice_treino], X_treino.iloc[indice_valida]
#         y_treino_split, y_valida_split = y_treino.iloc[indice_treino], y_treino.iloc[indice_valida]


#         # Aplica a transformação nas variáveis categóricas e numéricas
#         X_treino_split_transformado = preprocessor.fit_transform(X_treino_split)
#         X_valida_split_transformado = preprocessor.transform(X_valida_split)  # Não aplicamos fit no conjunto de validação

#         # Aplica o SMOTE somente no conjunto de treino
#         X_treino_smote, y_treino_smote = smote.fit_resample(X_treino_split_transformado, y_treino_split)

#         # Treina o modelo
#         pipeline.fit(X_treino_smote, y_treino_smote)

        
#         # Avalia o modelo no treino
#         y_pred_treino = pipeline.predict(X_treino_smote)
#         recall_treino = recall_score(y_treino_smote, y_pred_treino, average='binary')
#         recall_treino_lista.append(recall_treino)
        
#         # Avalia o modelo no teste (validação)
#         y_pred_valida = pipeline.predict(X_valida_split_transformado)
#         recall_teste = recall_score(y_valida_split, y_pred_valida, average='binary')
#         recall_teste_lista.append(recall_teste)
    
#     # Calcula a média dos recalls
#     recall_treino_medio = np.mean(recall_treino_lista)
#     recall_teste_medio = np.mean(recall_teste_lista)
    
#     # Armazena os resultados
#     resultado_experimentos["estimador"].append(nome_modelo)
#     resultado_experimentos["recall_treino"].append(recall_treino_medio)
#     resultado_experimentos["recall_teste"].append(recall_teste_medio)
    
#     print(f'Treinamento do modelo {nome_modelo} finalizado')



# # Resultado final em DataFrame
# df_resultados = pd.DataFrame(resultado_experimentos)

# # Em casos de underfit, calcula a diferença entre o recall de treino e teste
# df_resultados["gap"] = (df_resultados["recall_treino"] - df_resultados["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# # Ordena os resultados pelo recall no teste
# df_resultados = df_resultados.sort_values("recall_teste", ascending=False)

# df_resultados

In [13]:
# Testando hiperparâmetros para não overfittar

In [14]:
# # Testando hiperparâmetos para ver se melhora o resultado

# # Parâmetros para busca no CatBoost
# param_grid = {
#     'iterations': [100, 200, 300],
#     'depth': [4, 6, 8],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'l2_leaf_reg': [1, 3, 5],
#     'border_count': [32, 64, 128]}

# # Configuração do cross-validation
# kf = StratifiedKFold(n_splits=5)

# # Configuração do modelo CatBoost
# catboost_model = CatBoostClassifier(verbose=0, random_state=42, auto_class_weights='Balanced')

# # Configuração do RandomizedSearchCV com 5 folds
# random_search = RandomizedSearchCV(
#     estimator=catboost_model,
#     param_distributions=param_grid,
#     n_iter=20,  # número de combinações aleatórias a testar
#     scoring='recall',
#     cv=kf,
#     random_state=42,
#     n_jobs=-1
# )

# # Ajuste do RandomizedSearchCV aos dados de treino
# random_search.fit(X_treino, y_treino)

# # Extrai os melhores parâmetros e o melhor recall
# melhores_parametros = random_search.best_params_
# melhor_recall = random_search.best_score_

# print(f"Melhores Parâmetros: {melhores_parametros}")
# print(f"Melhor Recall médio nos folds: {melhor_recall}")

# # Opcional: Avaliação final do modelo com os melhores parâmetros no conjunto de teste
# catboost_melhor = random_search.best_estimator_
# y_pred_teste = catboost_melhor.predict(X_teste)
# recall_teste = recall_score(y_teste, y_pred_teste)

# print(f"Recall na validação: {melhor_recall}")
# print(f"Recall no teste: {recall_teste}")

In [15]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
# from sklearn.metrics import recall_score
# import numpy as np
# import pandas as pd

# # Configuração dos hiperparâmetros para o XGBoost
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'colsample_bytree': [0.3, 0.7, 1.0],
#     'scale_pos_weight': [1, 2, 5, 10]
# }

# # Configuração do cross-validation
# kf = StratifiedKFold(n_splits=5)

# # Configuração do modelo XGBoost
# xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# # Configuração do RandomizedSearchCV com 5 folds e acompanhamento de progresso
# random_search = RandomizedSearchCV(
#     estimator=xgb_model,
#     param_distributions=param_grid,
#     n_iter=20,
#     scoring='recall',
#     cv=kf,
#     random_state=42,
#     n_jobs=-1,
#     verbose=10  # Exibe o progresso dos testes
# )

# # Ajuste do RandomizedSearchCV aos dados de treino
# random_search.fit(X_treino, y_treino)

# # Extrai os melhores parâmetros e o melhor recall
# melhores_parametros = random_search.best_params_
# melhor_recall = random_search.best_score_

# print(f"Melhores Parâmetros: {melhores_parametros}")
# print(f"Melhor Recall médio nos folds: {melhor_recall:.4f}")

# # Avaliação final do modelo com os melhores parâmetros no conjunto de teste
# xgb_melhor = random_search.best_estimator_
# y_pred_teste = xgb_melhor.predict(X_teste)
# recall_teste = recall_score(y_teste, y_pred_teste)

# print(f"Recall na validação: {melhor_recall:.4f}")
# print(f"Recall no teste: {recall_teste:.4f}")


In [16]:
# # Definição do grid de hiperparâmetros para o LightGBM
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'num_leaves': [20, 31, 40],
#     'max_depth': [-1, 10, 20, 30],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.7, 0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'scale_pos_weight': [1, 2, 5, 10]
# }

# # Configuração do cross-validation
# kf = StratifiedKFold(n_splits=5)

# # Configuração do modelo LightGBM
# lgbm_model = LGBMClassifier(random_state=42)

# # Configuração do RandomizedSearchCV com 5 folds e acompanhamento de progresso
# random_search = RandomizedSearchCV(
#     estimator=lgbm_model,
#     param_distributions=param_grid,
#     n_iter=20,  # Número de combinações de parâmetros a serem testadas
#     scoring='recall',
#     cv=kf,
#     random_state=42,
#     n_jobs=-1,
#     verbose=10  # Exibe o progresso dos testes
# )

# # Ajuste do RandomizedSearchCV aos dados de treino
# random_search.fit(X_treino, y_treino)

# # Extrai os melhores parâmetros e o melhor recall
# melhores_parametros = random_search.best_params_
# melhor_recall = random_search.best_score_

# print(f"Melhores Parâmetros: {melhores_parametros}")
# print(f"Melhor Recall médio nos folds: {melhor_recall:.4f}")

# # Avaliação final do modelo com os melhores parâmetros no conjunto de teste
# lgbm_melhor = random_search.best_estimator_
# y_pred_teste = lgbm_melhor.predict(X_teste)
# recall_teste = recall_score(y_teste, y_pred_teste)

# print(f"Recall na validação: {melhor_recall:.4f}")
# print(f"Recall no teste: {recall_teste:.4f}")

In [17]:
# Hiperparâmetros
params_lgbm = {'subsample': 0.7, 'scale_pos_weight': 10, 'num_leaves': 20, 'n_estimators': 50, 'max_depth': -1, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
params_xgboost = {'scale_pos_weight': 10, 'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 1.0}
params_catboost = {'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 200, 'depth': 4, 'border_count': 128}

# Pipelines configurados com hiperparâmetros
pipe_catboost_balanceado_tunado = Pipeline([
    ('preprocessor', preprocessor),
    ("catboost", CatBoostClassifier(random_state=42, verbose=0, **params_catboost))
])

pipe_xgboost_tunado = Pipeline([
    ('preprocessor', preprocessor),
    ("xgboost", XGBClassifier(random_state=42, **params_xgboost))
])

pipe_lgbm_tunado = Pipeline([
    ('preprocessor', preprocessor),
    ("lgbm", LGBMClassifier(random_state=42, **params_lgbm))
])

# Pipelines configurados com hiperparâmetros e SMOTE:

pipe_catboost_balanceado_tunado_smote = Pipeline([
    ("catboost", CatBoostClassifier(random_state=42, verbose=0, **params_catboost))
])

pipe_xgboost_tunado_smote = Pipeline([
    ("xgboost", XGBClassifier(random_state=42, **params_xgboost))
])

pipe_lgbm_tunado_smote = Pipeline([
    ("lgbm", LGBMClassifier(random_state=42, **params_lgbm))
])

# Dicionário de pipelines

dict_pipes_tunado= {
    "CatBoost balanceado tunado": pipe_catboost_balanceado_tunado,
    "XGBoost balanceado tunado": pipe_xgboost_tunado,
    "LGBM balanceado tunado": pipe_lgbm_tunado
}


dict_pipes_tunado_smote = {
    "CatBoost balanceado tunado SMOTE": pipe_catboost_balanceado_tunado_smote,
    "XGBoost balanceado tunado SMOTE": pipe_xgboost_tunado_smote,
    "LGBM balanceado tunado SMOTE": pipe_lgbm_tunado
}

In [18]:
# Mesma coisa usando CV e tunagem

# Configuração do StratifiedKFold
kf = StratifiedKFold(n_splits=5)

# Inicialização do dicionário de resultados
resultado_experimentos = {"estimador": [], "recall_treino": [], "recall_teste": []}

# Loop pelos pipelines
for nome_modelo, pipeline in dict_pipes_tunado.items():
    recall_treino_lista = []
    recall_teste_lista = []
    
    # Realiza a validação cruzada estratificada
    for indice_treino, indice_valida in kf.split(X_treino, y_treino):
        X_treino_split, X_valida_split = X_treino.iloc[indice_treino], X_treino.iloc[indice_valida]
        y_treino_split, y_valida_split = y_treino.iloc[indice_treino], y_treino.iloc[indice_valida]
        
        # Treina o modelo
        pipeline.fit(X_treino_split, y_treino_split)
        
        # Avalia o modelo no treino
        y_pred_treino = pipeline.predict(X_treino_split)
        recall_treino = recall_score(y_treino_split, y_pred_treino, average='binary')
        recall_treino_lista.append(recall_treino)
        
        # Avalia o modelo no teste (validação)
        y_pred_valida = pipeline.predict(X_valida_split)
        recall_teste = recall_score(y_valida_split, y_pred_valida, average='binary')
        recall_teste_lista.append(recall_teste)
    
    # Calcula a média dos recalls
    recall_treino_medio = np.mean(recall_treino_lista)
    recall_teste_medio = np.mean(recall_teste_lista)
    
    # Armazena os resultados
    resultado_experimentos["estimador"].append(nome_modelo)
    resultado_experimentos["recall_treino"].append(recall_treino_medio)
    resultado_experimentos["recall_teste"].append(recall_teste_medio)
    
    print(f'Treinamento do modelo {nome_modelo} finalizado')

# Resultado final em DataFrame
df_resultados = pd.DataFrame(resultado_experimentos)

# Em casos de underfit, calcula a diferença entre o recall de treino e teste
df_resultados["gap"] = (df_resultados["recall_treino"] - df_resultados["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# Ordena os resultados pelo recall no teste
df_resultados = df_resultados.sort_values("recall_teste", ascending=False)

df_resultados

Treinamento do modelo CatBoost balanceado tunado finalizado
Treinamento do modelo XGBoost balanceado tunado finalizado
[LightGBM] [Info] Number of positive: 1670, number of negative: 10057
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000307 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 858
[LightGBM] [Info] Number of data points in the train set: 11727, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142406 -> initscore=-1.795445
[LightGBM] [Info] Start training from score -1.795445
[LightGBM] [Info] Number of positive: 1670, number of negative: 10057
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001064 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 853
[LightGBM] [Info] Number of data points in the train set: 11

,estimador,recall_treino,recall_teste,gap
1,XGBoost balanceado tunado,0.905247,0.898910,0.006337
2,LGBM balanceado tunado,0.918303,0.847644,0.070659
0,CatBoost balanceado tunado,0.699690,0.687607,0.012083


In [ ]:
# # Mesma coisa usando CV e SMOTE e tunage

# # Configuração do StratifiedKFold
# kf = StratifiedKFold(n_splits=5)

# # Inicialização do dicionário de resultados
# resultado_experimentos = {"estimador": [], "recall_treino": [], "recall_teste": []}

# smote = SMOTE(random_state=42, n_jobs=-1)

# X = df.drop(columns=['status_emprestimo'], axis=1)
# y = df['status_emprestimo']

# X_teste, X_treino, y_teste, y_treino = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y, shuffle=True)


# # Loop pelos pipelines
# for nome_modelo, pipeline in dict_pipes_tunado_smote.items():
#     recall_treino_lista = []
#     recall_teste_lista = []
    
#     # Realiza a validação cruzada estratificada
#     for indice_treino, indice_valida in kf.split(X_treino, y_treino):
#         X_treino_split, X_valida_split = X_treino.iloc[indice_treino], X_treino.iloc[indice_valida]
#         y_treino_split, y_valida_split = y_treino.iloc[indice_treino], y_treino.iloc[indice_valida]


#         # Aplica a transformação nas variáveis categóricas e numéricas
#         X_treino_split_transformado = preprocessor.fit_transform(X_treino_split)
#         X_valida_split_transformado = preprocessor.transform(X_valida_split)  # Não aplicamos fit no conjunto de validação

#         # Aplica o SMOTE somente no conjunto de treino
#         X_treino_smote, y_treino_smote = smote.fit_resample(X_treino_split_transformado, y_treino_split)

#         # Treina o modelo
#         pipeline.fit(X_treino_smote, y_treino_smote)

        
#         # Avalia o modelo no treino
#         y_pred_treino = pipeline.predict(X_treino_smote)
#         recall_treino = recall_score(y_treino_smote, y_pred_treino, average='binary')
#         recall_treino_lista.append(recall_treino)
        
#         # Avalia o modelo no teste (validação)
#         y_pred_valida = pipeline.predict(X_valida_split_transformado)
#         recall_teste = recall_score(y_valida_split, y_pred_valida, average='binary')
#         recall_teste_lista.append(recall_teste)
    
#     # Calcula a média dos recalls
#     recall_treino_medio = np.mean(recall_treino_lista)
#     recall_teste_medio = np.mean(recall_teste_lista)
    
#     # Armazena os resultados
#     resultado_experimentos["estimador"].append(nome_modelo)
#     resultado_experimentos["recall_treino"].append(recall_treino_medio)
#     resultado_experimentos["recall_teste"].append(recall_teste_medio)
    
#     print(f'Treinamento do modelo {nome_modelo} finalizado')



# # Resultado final em DataFrame
# df_resultados = pd.DataFrame(resultado_experimentos)

# # Em casos de underfit, calcula a diferença entre o recall de treino e teste
# df_resultados["gap"] = (df_resultados["recall_treino"] - df_resultados["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# # Ordena os resultados pelo recall no teste
# df_resultados = df_resultados.sort_values("recall_teste", ascending=False)

# df_resultados